In [1]:
import h5py
import os
%pylab nbagg

Populating the interactive namespace from numpy and matplotlib


In [2]:
%pylab nbagg
import h5py
import os
import matplotlib.pyplot as plt
import pandas as pd

import scipy 
import scipy.fftpack
import seaborn as sns

from scipy.fft import fft, ifft
from scipy import signal

from scipy import stats

Populating the interactive namespace from numpy and matplotlib


In [3]:
os.chdir("C:/Users/celie/OneDrive - ugentbe/NOG NODIG/2de master/Masterproef/Datasets/All_freqs")
mydir = "C:/Users/celie/OneDrive - ugentbe/NOG NODIG/2de master/Masterproef/Datasets/All_freqs"

In [4]:
import pandas as pd 
filename = mydir + "/Health_ROI_order.csv"
ROI_H = pd.read_csv(filename)
ROI_H = np.array(ROI_H)

filename = mydir + "/Depr_ROI_order.csv"
ROID = pd.read_csv(filename)
ROI_D = np.array(ROID)

filename = mydir + "//HF_ROI_order.csv"
ROIHF = pd.read_csv(filename)
ROI_HF = np.array(ROIHF)

filename = mydir + "/LF_ROI_order.csv"
ROILF = pd.read_csv(filename)
ROI_LF = np.array(ROILF)

In [5]:
filename = mydir + "/Depr_coh_total_hstacked100.h5"
with h5py.File(filename, "r") as f:
     Depr = f['data'][:]

In [6]:
filename = mydir + "/Health_coh_total_hstacked100.h5"
with h5py.File(filename, "r") as f:
     Health = f['data'][:]

In [7]:
filename = mydir + "/HF_coh_total_hstacked100.h5"
with h5py.File(filename, "r") as f:
     HF = f['data'][:]

In [8]:
filename = mydir + "/LF_coh_total_hstacked100.h5"
with h5py.File(filename, "r") as f:
     LF = f['data'][:]

In [9]:
print(shape(Depr))
print(shape(Health))
print(shape(HF))
print(shape(LF))

(378, 500)
(378, 500)
(378, 500)
(378, 500)


In [10]:
print(shape(ROI_D))
print(shape(ROI_H))
print(shape(ROI_HF))
print(shape(ROI_LF))

(378, 4)
(378, 4)
(378, 4)
(378, 4)


In [11]:
import statistics
def EFsize(c0, c1):
    d = (mean(c0) - mean(c1)) / (sqrt((statistics.stdev(c0) ** 2 + statistics.stdev(c1) ** 2) / 2))
    return d

In [12]:
c1 = np.repeat("", 1)
c2 = np.repeat("",1)

n_pairs = len(ROI_D)
sig_count = False 
base = numpy.zeros(shape=(1,20)) 
#ROI1, ROI2, mean (4 times), sd (4 times), p value (H_D, H_HF, H_LF, D_HF, D_LF, HF_LF)
sign_array = np.column_stack([c1, c2, base])
sign_array_tot= np.column_stack([c1, c2, base])

p_array = np.column_stack([c1, c2,base])
p_array_tot= np.column_stack([c1, c2, base])

for i in range(n_pairs):
    data_dep = Depr[i]
    data_health = Health[i]
    data_HF = HF[i]
    data_LF = LF[i]
    
    ROI1 = ROI_D[i][0]
    ROI2 = ROI_D[i][1]
    
    #p_values
    p_H_D = stats.ttest_ind(data_dep, data_health).pvalue
    p_H_LF = stats.ttest_ind(data_LF, data_health).pvalue
    p_H_HF = stats.ttest_ind(data_HF, data_health).pvalue
    p_D_LF = stats.ttest_ind(data_dep, data_LF).pvalue
    p_D_HF = stats.ttest_ind(data_dep, data_HF).pvalue
    p_HF_LF = stats.ttest_ind(data_LF, data_HF).pvalue
    
    #Effect sizes
    EF_H_D = EFsize(data_dep, data_health)
    EF_H_LF = EFsize(data_LF, data_health)
    EF_H_HF = EFsize(data_HF, data_health)
    EF_D_LF = EFsize(data_dep, data_LF)
    EF_D_HF = EFsize(data_dep, data_HF)
    EF_HF_LF = EFsize(data_LF, data_HF)
    
    #make general p_value output (for all comparisons)
    p_array[0][0] = str(ROI1)
    p_array[0][1] = str(ROI2)
    p_array[0][2] = mean(data_dep)
    p_array[0][3] = mean(data_health)
    p_array[0][4] = mean(data_HF)
    p_array[0][5] = mean(data_LF)
    p_array[0][6] = std(data_dep)
    p_array[0][7] = std(data_health)
    p_array[0][8] = std(data_HF)
    p_array[0][9] = std(data_LF)
    p_array[0][10] = p_H_D
    p_array[0][11] = p_H_LF
    p_array[0][12] = p_H_HF
    p_array[0][13] = p_D_LF
    p_array[0][14] = p_D_HF
    p_array[0][15] = p_HF_LF
    p_array[0][16] = EF_H_D
    p_array[0][17] = EF_H_LF
    p_array[0][18] = EF_H_HF
    p_array[0][19] = EF_D_LF
    p_array[0][20] = EF_D_HF
    p_array[0][21] = EF_HF_LF
    
    
    
    
    #if any of p-values is significant: store these regions and all p-values corresponding in the significant matrix
    if p_H_D <= 0.05 or p_H_LF <= 0.05 or p_H_HF <= 0.05 or p_D_LF <= 0.05 or p_D_HF <= 0.05 or p_HF_LF <= 0.05:
        sign_array[0][0] = str(ROI1)
        sign_array[0][1] = str(ROI2)
        sign_array[0][2] = mean(data_dep)
        sign_array[0][3] = mean(data_health)
        sign_array[0][4] = mean(data_HF)
        sign_array[0][5] = mean(data_LF)
        sign_array[0][6] = std(data_dep)
        sign_array[0][7] = std(data_health)
        sign_array[0][8] = std(data_HF)
        sign_array[0][9] = std(data_LF)
        sign_array[0][10] = p_H_D
        sign_array[0][11] = p_H_LF
        sign_array[0][12] = p_H_HF
        sign_array[0][13] = p_D_LF
        sign_array[0][14] = p_D_HF
        sign_array[0][15] = p_HF_LF
        sign_array[0][16] = EF_H_D
        sign_array[0][17] = EF_H_LF
        sign_array[0][18] = EF_H_HF
        sign_array[0][19] = EF_D_LF
        sign_array[0][20] = EF_D_HF
        sign_array[0][21] = EF_HF_LF
        
        if sig_count == False :
            sign_array_tot = sign_array
            sig_count = True 
        else:
            tupley = (sign_array_tot, sign_array)
            sign_array_tot = np.vstack(tupley)
 
    
    if i == 0:
        p_array_tot = p_array
    else:
        tuplex = (p_array_tot, p_array)
        p_array_tot = np.vstack(tuplex)

        
print("ok")     
file_name = mydir + "/significance_coherence_EF.csv"
outcome = pd.DataFrame.from_records(sign_array_tot)
outcome.columns = ["ROI1", "ROI2", "meanDEP", "meanHEALTH", "meanHF", "meanLF","stdDEP", "stdHEALTH","stdHF","stdLF","p HD",
                  "p H_LF", "p H_HF","p D_LF", "p D_HF",'p_HF_LF',"EF HD",
                  "EF H_LF", "EF H_HF","EF D_LF", "EF D_HF",'EF_HF_LF']
outcome.to_csv(path_or_buf = file_name, index = False)
        
    
file_name = mydir + "/all_p-values_coherence_EF.csv"
outcome = pd.DataFrame.from_records(p_array_tot)
outcome.columns = ["ROI1", "ROI2", "meanDEP", "meanHEALTH", "meanHF", "meanLF","stdDEP", "stdHEALTH","stdHF","stdLF","p HD",
                  "p H_LF", "p H_HF","p D_LF", "p D_HF",'p_HF_LF',"EF HD",
                  "EF H_LF", "EF H_HF","EF D_LF", "EF D_HF",'EF_HF_LF']
outcome.to_csv(path_or_buf = file_name, index = False)

ok


In [13]:
from rpy2.robjects.packages import importr
from rpy2.robjects.vectors import FloatVector

stats = importr('stats')

In [14]:
#calculate adjusted p-values for all values

p_array_tot_corrected = np.copy(p_array_tot)

pHD = p_array_tot[:,10] #p values between depressed and healthy
pH_HF = p_array_tot[:,11]
pH_LF = p_array_tot[:,12]
pD_HF =p_array_tot[:,13]
pD_LF =p_array_tot[:,14]
pHF_LF=p_array_tot[:,15]

pHD = pHD.reshape(378,1)
pH_HF = pH_HF.reshape(378,1)
pH_LF = pH_LF.reshape(378,1)
pD_HF = pD_HF.reshape(378,1)
pD_LF= pD_LF.reshape(378,1)
pHF_LF= pHF_LF.reshape(378,1)

p_adjust_H_D = stats.p_adjust(FloatVector(pHD), method = 'holm')
p_adjust_H_HF = stats.p_adjust(FloatVector(pH_HF), method = 'holm')
p_adjust_H_LF = stats.p_adjust(FloatVector(pH_LF), method = 'holm')
p_adjust_D_HF = stats.p_adjust(FloatVector(pD_HF), method = 'holm')
p_adjust_D_LF = stats.p_adjust(FloatVector(pD_LF), method = 'holm')
p_adjust_LH_HF = stats.p_adjust(FloatVector(pHF_LF), method = 'holm')



In [15]:
print(p_array_tot[:,12])
print(p_array_tot_corrected[:,12])

['0.9980497947864928' '0.9980497947864928' '0.9947519756451167'
 '0.9987562114941079' '0.49631278843167537' '0.9986977225334753'
 '0.9977706714031477' '0.3927215989897528' '0.9979876862211471'
 '0.9987939987814657' '0.9983825935858254' '0.403439005896041'
 '0.6436885164650683' '0.9983189770712024' '0.9991995737148841'
 '0.9981395840869681' '0.9974993486798829' '0.9967770654240601'
 '0.09396701581379105' '0.9990884308349484' '0.9975607656821255'
 '0.20835593677546407' '0.9972310287271384' '0.9980271441721092'
 '0.998191519126805' '0.43296864607380403' '0.40744061560412126'
 '0.9987137972193022' '0.9977738501684731' '0.9978653150844878'
 '0.1789676458340552' '0.9995863223984126' '0.9993335680302627'
 '0.4686174896355827' '0.9981020455841192' '0.99854028499558'
 '0.9988720585919542' '0.863172982816781' '0.5386967400564571'
 '0.9986462807718968' '0.9993716351737982' '0.9950810917680355'
 '0.9967029411834576' '0.9975861319345748' '0.8317612048506927'
 '0.9992766180100026' '0.996854749479100

In [16]:
p_array_tot_corrected[:,10] = p_adjust_H_D 
p_array_tot_corrected[:,11] =p_adjust_H_HF
p_array_tot_corrected[:,12] = p_adjust_H_LF
p_array_tot_corrected[:,13] = p_adjust_D_HF
p_array_tot_corrected[:,14] = p_adjust_D_LF
p_array_tot_corrected[:,15] =p_adjust_LH_HF

In [18]:
tuple = (p_array_tot_corrected, p_array_tot)
array_complete = np.hstack(tuple)

file_name = mydir + "/all_p-values_coherence_EF_corrected.csv"
outcome = pd.DataFrame.from_records(array_complete)
outcome.columns = ["ROI1", "ROI2", "meanDEP", "meanHEALTH", "meanHF", "meanLF","stdDEP", "stdHEALTH","stdHF","stdLF","padj HD",
                  "padj H_LF", "padj H_HF","padj D_LF", "padj D_HF",'padj_HF_LF',"EF HD",
                  "EF H_LF", "EF H_HF","EF D_LF", "EF D_HF",'EF_HF_LF',
                  "ROI1", "ROI2", "meanDEP", "meanHEALTH", "meanHF", "meanLF","stdDEP", "stdHEALTH","stdHF","stdLF","p HD",
                  "p H_LF", "p H_HF","p D_LF", "p D_HF",'p_HF_LF',"EF HD",
                  "EF H_LF", "EF H_HF","EF D_LF", "EF D_HF",'EF_HF_LF']
outcome.to_csv(path_or_buf = file_name, index = False)

In [ ]:
print(shape(p_array_tot))
print(shape(p_array_tot_corrected))

In [ ]:
p_adjust_H_HF = np.array(p_adjust_H_HF)

In [ ]:
p_adjust_H_HF